# Summary

# Imports

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
pd.set_option("max_columns", 100)

# Parameters

In [ ]:
NOTEBOOK_PATH = Path('train_network')
NOTEBOOK_PATH

In [ ]:
OUTPUT_PATH = Path(os.getenv('OUTPUT_DIR', NOTEBOOK_PATH.name)).resolve()
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH

In [ ]:
DEBUG = "CI" not in os.environ
if DEBUG:
    assert "SLURM_JOB_ID" not in os.environ
DEBUG

# Train network

In [ ]:
DATAPKG_OUTPUT_DIR = os.environ['DATAPKG_OUTPUT_DIR']
DATAPKG_OUTPUT_DIR

In [ ]:
if DEBUG:
    RUNTIME = '1m'
else:
    RUNTIME = os.environ['RUNTIME']
RUNTIME

In [ ]:
!python -m pagnn.training.dcn \
    --root-path {OUTPUT_PATH} \
    --training-data-path {DATAPKG_OUTPUT_DIR}/adjacency-net-v2/master/training_dataset/adjacency_matrix.parquet \
    --gpu -1 \
    --verbosity 1 \
    --runtime {RUNTIME} \